In [110]:
import ee,requests,os,sys,shutil,threading
from IPython.display import Image
ee.Initialize(project='lcms-292214',)

import geeViz.getImagesLib as gil
import geeViz.changeDetectionLib as cdl
Map = gil.Map

output_folder = '../assets/images'
version = '2024-10'
conus_loss = os.path.join(output_folder,f'CONUS_v{version}_Change_Loss.png')
conus_loss_gain = os.path.join(output_folder,f'CONUS_v{version}_Change_Loss_Gain.png')
conus_change = os.path.join(output_folder,f'CONUS_v{version}_Change.png')
conus_lc = os.path.join(output_folder,f'CONUS_v{version}_Land_Cover.png')
conus_lc3 = os.path.join(output_folder,f'CONUS_v{version}_Land_Cover3.png')
conus_lu = os.path.join(output_folder,f'CONUS_v{version}_Land_Use.png')

In [108]:
viz = {
    "Change_class_names": [
        "Wind",
        "Hurricane",
        "Snow or Ice Transition",
        "Desiccation",
        "Inundation",
        "Prescribed Fire",
        "Wildfire",
        "Mechanical Land Transformation",
        "Tree Removal",
        "Defoliation",
        "Southern Pine Beetle",
        "Insect, Disease, or Drought Stress",
        "Other Loss",
        "Vegetation Successional Growth",
        "Stable",
        "Non-Processing Area Mask"
    ],
    "Change_class_palette": [
        "FF09F3",
        "541AFF",
        "E4F5FD",
        "CC982E",
        "0ADAFF",
        "A10018",
        "D54309",
        "FAFA4B",
        "AFDE1C",
        "FFC80D",
        "A64C28",
        "F39268",
        "C291D5",
        "00A398",
        "3D4551",
        "1B1716"
    ],
    "Change_class_values": [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16
    ],
    
    
    "Land_Cover3_class_names": [
        "Tree",
        "Shrub",
        "Grass/Forb/Herb",
        "Barren or Impervious",
        "Snow or Ice",
        "Water",
        "Non-Processing Area Mask"
    ],
    "Land_Cover3_class_palette": [
        "004E2B",
        "F89A1C",
        "E5E98A",
        "893F54",
        "E4F5FD",
        "00B6F0",
        "1B1716"
    ],
    "Land_Cover3_class_values": [
        1,
        2,
        3,
        4,
        5,
        6,
        7
    ],

    "Land_Cover_class_names": [
        "Tree",
        "Tall Shrub & Tree Mix (AK Only)",
        "Shrub & Tree Mix",
        "Grass/Forb/Herb & Tree Mix",
        "Barren & Tree Mix",
        "Tall Shrub (AK Only)",
        "Shrub",
        "Grass/Forb/Herb & Shrub Mix",
        "Barren & Shrub Mix",
        "Grass/Forb/Herb",
        "Barren & Grass/Forb/Herb Mix",
        "Barren or Impervious",
        "Snow or Ice",
        "Water",
        "Non-Processing Area Mask"
    ],
    "Land_Cover_class_palette": [
        "004E2B",
        "009344",
        "61BB46",
        "ACBB67",
        "8B8560",
        "CAFD4B",
        "F89A1C",
        "8FA55F",
        "BEBB8E",
        "E5E98A",
        "DDB925",
        "893F54",
        "E4F5FD",
        "00B6F0",
        "1B1716"
    ],
    "Land_Cover_class_values": [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15
    ]
,
    
    "Land_Use_class_names": [
        "Agriculture",
        "Developed",
        "Forest",
        "Other",
        "Rangeland or Pasture",
        "Non-Processing Area Mask"
    ],
    "Land_Use_class_palette": [
        "FBFF97",
        "E6558B",
        "004E2B",
        "9DBAC5",
        "A6976A",
        "1B1716"
    ],
    "Land_Use_class_values": [
        1,
        2,
        3,
        4,
        5,
        6
    ]


}

In [107]:
# Get CONUS Loss and Loss+Gain images


print(conus_loss)
gil.vizParamsTrue = {'min': 0.1, 'max': [0.3, 0.3, 0.3], 'bands': 'red,green,blue'}
sa = ee.FeatureCollection("projects/lcms-tcc-shared/assets/CONUS/Ancillary/Conus_NLCD_Boundary")

crs = gil.common_projections['NLCD_CONUS']['crs']
transform = gil.common_projections['NLCD_CONUS']['transform']
print(sa.geometry().bounds(500,crs).coordinates().getInfo())
dem = ee.Image("USGS/3DEP/10m").resample("bicubic")
hillshade = ee.Terrain.hillshade(dem)
# transform[0] = 300
# transform[-2] = 300
print(transform)

def getMostRecentChange(c, code,msk):
    c = c.select(['Change'])
    def wrapper(img):
        yr = ee.Date(img.get("system:time_start")).get("year")
        out = ee.Image(yr).int16().updateMask(msk)
        out = out.where(img.neq(code),0).setDefaultProjection(crs,transform)#.selfMask()
        return (
            ee.Image(out
            .rename(["year"])
            .copyProperties(img, ["system:time_start"])).reduceResolution(ee.Reducer.firstNonNull(),True,256).selfMask()#.reproject(crs,transform)#.selfMask()
        )

    return c.map(wrapper)
def downloadImage(url,filename):
    img_data = requests.get(url).content
    with open(filename, 'wb') as handler:
        handler.write(img_data)

comp = ee.ImageCollection("projects/lcms-tcc-shared/assets/CONUS/Composites/Composite-Collection-yesL7")\
.filter(ee.Filter.calendarRange(2021,2024,'year')).mean()#.unmask()#.resample('bicubic').reproject(crs,transform)
gil.vizParamsTrue10k = {'min': 100, 'max': [5000.0, 5000.0, 4400.0], 'bands': 'red,green,blue',}
Map.clearMap()
Map.addLayer(comp,gil.vizParamsFalse10k,'Comp Raw')

lcms = ee.ImageCollection("projects/lcms-292214/assets/Final_Outputs/2024-10/LCMS").filter("study_area=='CONUS'")\
.map(lambda img:img.set(viz))
print(lcms.first().getInfo())

msk = lcms.first().select([0]).mask()
msk = msk.focal_min(1.5,'circle','pixels')

comp = comp.updateMask(msk)
comp = comp.visualize(**gil.vizParamsTrue10k)#.setDefaultProjection(crs,transform)#.reduceResolution(ee.Reducer.mean(),True).reproject(crs,transform)


Map.addLayer(comp,{},'Comp Raw Contracted')
Map.addLayer(msk,{},'MSK')

props = lcms.first().toDictionary()
lc = lcms.select(['Land_Cover']).sort('system:time_start',False).first().set(props).setDefaultProjection(crs,transform).visualize()
lu = lcms.select(['Land_Use']).sort('system:time_start',False).first().set(props).setDefaultProjection(crs,transform).visualize()

all_change = lcms.select(['Change']).min()

all_change = all_change.setDefaultProjection(crs,transform)
change_msk = all_change.lte(14)
all_change = all_change.updateMask(change_msk).reduceResolution(ee.Reducer.min(),True,8).updateMask(change_msk.focal_max(1).And(msk))#
all_change = all_change.set(props).visualize()

all_change_msk = all_change.mask()
all_change = all_change.unmask(comp)
Map.addLayer(all_change,{},'All Change')
# fast_loss = getMostRecentChange(lcms,3,msk).mosaic().visualize(**{'min':1985,'max':2023,'palette':cdl.lossYearPalette})
# slow_loss = getMostRecentChange(lcms,2,msk).mosaic().visualize(**{'min':1985,'max':2023,'palette':cdl.lossYearPalette})
# gain = getMostRecentChange(lcms,4,msk).mosaic().visualize(**{'min':1985,'max':2023,'palette':cdl.gainYearPalette})

# loss = comp
# loss = loss.where(fast_loss.neq(0),fast_loss)
# loss = loss.where(slow_loss.neq(0),slow_loss)

# loss_gain = comp

# loss_gain = loss_gain.where(fast_loss.neq(0),fast_loss)
# loss_gain = loss_gain.where(slow_loss.neq(0),slow_loss)
# loss_gain = loss_gain.where(gain.neq(0),gain)

# # loss = hillshade.blend(loss.visualize(**{"opacity": 0.7}))
# # loss_gain = hillshade.blend(loss_gain.visualize(**{"opacity": 0.7}))

# Map.addLayer(loss,{},'Loss')
# Map.addLayer(loss_gain,{},'Loss+Gain')
# Map.addLayer(lc,{},'Land Cover')
# Map.addLayer(lu,{},'Land Use')
Map.turnOnInspector()
# Map.view()

params = {
  'region': sa.geometry().bounds(500,crs),
  'dimensions': 1000,
  'crs': crs,
  'transform':transform
}

# Get a URL that will generate a GIF.
# url = comps.getVideoThumbURL(params)

all_change_url = all_change.getThumbURL(params)
# loss_url = loss.getThumbURL(params)
t = threading.Thread(target = downloadImage,args=(all_change_url,conus_change))
t.start()

# params = {
#   'region': sa.geometry().bounds(500,crs),
#   'dimensions': 250,
#   'crs': crs,
#   'transform':transform
# }
# loss_gain_url = loss_gain.getThumbURL(params)
# t = threading.Thread(target = downloadImage,args=(loss_gain_url,conus_loss_gain))
# t.start()

# print(loss_url,loss_gain_url)
print(all_change_url)

# display(Image(url=loss_url))
# display(Image(url=loss_gain_url))
display(Image(url=all_change_url))



../assets/images\CONUS_v2024-10_Change_Loss.png
[[[-2361585.0204712525, 259064.86686735478], [2263785.109653341, 259064.86686735478], [2263785.109653341, 3177404.865907416], [-2361585.0204712525, 3177404.865907416], [-2361585.0204712525, 259064.86686735478]]]
[30, 0, -2361915.0, 0, -30, 3177735.0]
Adding layer: Comp Raw
{'type': 'Image', 'bands': [{'id': 'Change', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [196082, 104128], 'crs': 'PROJCS["Albers_Conical_Equal_Area", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      TOWGS84[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Albers_Conic_Equal_Area"], \n  PARAMETER["central_meri

In [45]:
print(threading.active_count())

7


In [112]:

params = {
  'region': sa.geometry().bounds(500,crs),
  'dimensions': 1000,
  'crs': crs,
  'transform':transform
}

lc = lcms.select(['Land_Cover']).sort('system:time_start',False).first()

lc3 = lc.remap([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],[1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 5, 6, 7]).rename(['Land_Cover3']).set(viz).setDefaultProjection(crs,transform).visualize()

lc = lc.set(viz).setDefaultProjection(crs,transform).visualize()
lu = lcms.select(['Land_Use']).sort('system:time_start',False).first().set(props).setDefaultProjection(crs,transform).visualize()

lc_url = lc.getThumbURL(params)
lc3_url = lc3.getThumbURL(params)
lu_url = lu.getThumbURL(params)

t = threading.Thread(target = downloadImage,args=(lc_url,conus_lc))
t.start()
t = threading.Thread(target = downloadImage,args=(lc3_url,conus_lc3))
t.start()
t = threading.Thread(target = downloadImage,args=(lu_url,conus_lu))
t.start()
print(lc_url,lu_url)

display(Image(url=lc_url))
display(Image(url=lc3_url))
display(Image(url=lu_url))



https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/4787f23a0f861c42afa69eb1ac9b4537-3dd4a00143181d853499992b4c95d3de:getPixels https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/2fb5f66aac766ec4c11a3db809b59045-8e5276c7fcbab3f1dc9537a48bddfa06:getPixels


In [4]:
# Move gifs
from_dir = r'Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs'

which_ones = ['Atlanta-Land_Use_wYears.gif','Great_Salt_Lake-Land_Cover_wYears.gif','Northern_Colorado-Change_wYears.gif','Columbia_Glacier-Land_Cover_wYears.gif','Lake_Mead-Land_Cover_wYears.gif','Oregon_Wildfire-Change_wYears.gif','Las_Vegas-Land_Use_wYears.gif','Fountain_Wildfire-Change_wYears.gif','Denver-Land_Use_wYears.gif']

for w in which_ones:
    print('Copying:',os.path.basename(w))
    input = os.path.join(from_dir,os.path.basename(w))
    
    output = os.path.join(output_folder,os.path.basename(w))
    print(input,output)
    shutil.copy2(input,output)

Copying: Atlanta-Land_Use_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Atlanta-Land_Use_wYears.gif ../assets/images\Atlanta-Land_Use_wYears.gif
Copying: Great_Salt_Lake-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Great_Salt_Lake-Land_Cover_wYears.gif ../assets/images\Great_Salt_Lake-Land_Cover_wYears.gif
Copying: Northern_Colorado-Change_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Northern_Colorado-Change_wYears.gif ../assets/images\Northern_Colorado-Change_wYears.gif
Copying: Columbia_Glacier-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Columbia_Glacier-Land_Cover_wYears.gif ../assets/images\Columbia_Glacier-Land_Cover_wYears.gif
Copying: Lake_Mead-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\